# Befood

## restaurant

In [ ]:
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import re
import json
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import math
import csv
import requests
from tqdm import tqdm

In [ ]:
befood_restaurant = r"./Grab/data/data_grb/befood/restaurant_info.csv"
befood_review = r"./Grab/data/data_grb/befood/reviews.csv"
befood_dishes = r"./Grab/data/data_grb/befood/dishes.csv"

befood_restaurant = pd.read_csv(befood_restaurant, encoding='utf-8')
befood_review = pd.read_csv(befood_review, encoding='utf-8')
befood_dishes = pd.read_csv(befood_dishes, encoding='utf-8')

In [ ]:
# remove rows with all NaN values in the restaurant_id column
befood_restaurant = befood_restaurant.dropna(subset=['restaurant_id'])
#show 'restaurant_id', 'restaurant_name', 'latitude', 'longitude', 'display_address', 'rating', 'review_count', 'city' columns
befood_restaurant = befood_restaurant[['restaurant_id', 'restaurant_name', 'latitude', 'longitude', 'display_address', 'rating', 'review_count', 'city']]
#rename columns 'display_address': 'address', 'rating': 'restaurant_rating', 'city': 'city_id'
befood_restaurant.rename(columns={'display_address': 'address', 'rating': 'restaurant_rating', 'city': 'city_id'}, inplace=True)
# add plateform_id = 0
befood_restaurant['platform_id'] = 0
befood_restaurant = befood_restaurant.dropna(subset=['restaurant_id'])

In [ ]:
# 1) Biên dịch pattern đầy đủ, loại bỏ các match không có nội dung sau "Quận"/"Q."
pattern = re.compile(
    r'('
      r'(?:Quận|Q\.)\s*[^\s\-,]+'             # Bắt "Quận" hoặc "Q." + từ đầu tiên KHÔNG được là dấu cách
      r'(?:\s+[^\s\-,]+)*'                    # Bắt thêm các token tiếp theo nếu có
      r'(?=,| - | Thành Phố| TP\.| TP | TP HCM| HCM| Hồ Chí Minh| HN| Hà Nội|$)'  # Dừng đúng chỗ
    r'|'
      r'(?:Thành Phố|TP\.?|Tp\.?|tp\.?)\s*Thủ Đức'  # Bắt Thành Phố Thủ Đức, TP Thủ Đức, ...
    r')',
    flags=re.IGNORECASE
)

def extract_district(address: str) -> str:
    if not isinstance(address, str):
        return None
    m = pattern.search(address)
    if m:
        return m.group(0).strip()
    return None

befood_restaurant['district_simple'] = befood_restaurant['address'].apply(extract_district)

In [ ]:
pd.set_option('display.max_colwidth', None)
# show district and address columns with none values in district column that are not null in address column
befood_restaurant[befood_restaurant['district_simple'].isna() & befood_restaurant['address'].notna()][['district_simple', 'address', 'longitude', 'latitude']].tail(10)

In [ ]:
normalize_map_full = {
    # TP. Hồ Chí Minh
    'quận 1': ['Quận 1. TPHCM', 'Quận 1 TP HCM','Quận 1 HCM', 'quận 1 thành phố Hồ Chí Minh', 'Quận 1 Hồ Chí Minh','quận 1','Quận 1', 'Q.1', 'Q. 1', 'Quận 01', 'Quận\xa01', 'Quận1'],
    'quận 2': ['Quận 2', 'Quận 2 (3 Đường 63)', 'Q.2', 'quận 2', 'Quận 2 Hồ Chí Minh','Q. 2'],
    'quận 3': ['Quận 3', 'Q. 3', 'Q.3', 'quận 3', 'Quận 3 Thành phố Hồ Chí Minh', 'Quận 3 TP. HCM','Quận\xa03', 'Quận 3. Hồ Chí Minh', 'Quận 3 Hồ Chí Minh','Quận 03'],
    'quận 4': ['Quận 4', 'Q. 4', 'Quận4)', 'Q.4', 'quận 4', 'Quận 04', 'Quận 4 Hồ Chí Minh', 'Quận\xa04', 'Quận 4. Hồ Chí Minh'],
    'quận 5': ['Quận 5', 'Q. 5', 'quận 5', 'Quận\xa05', 'Quận 5)', 'Quận 5 (đầu hẻm 327/71)', 'Quận 05', 'quận 5 TP HCM', 'Q.5', 'Quận 5 TP HCM'],
    'quận 6': ['Quận 6', 'Q.6', 'quận 6', 'Q. 6', 'Quận. 6', 'Quận 6)', 'Quận 06', 'Quận 6 Hồ Chí Minh', 'Quận\xa06'],
    'quận 7': ['Quận Tân Hưng', 'Quận Bình Thuận','Quận 7', 'Q. 7', 'Q.7', 'quận 7', 'quận 7 TP. HCM', 'Quận 7. Hồ Chí Minh', 'Quận 7.', 'Quận\xa07', 'Quận 7 Hồ Chí Minh', 'Quận  7'],
    'quận 8': ['Quận 8', 'Q.8', 'Q. 8', 'quận 8', 'Quận 08', 'Quận  8', 'Quận\xa08', 'Quận 8 Thành Phố Hồ Chí Minh'],
    'quận 9': ['Quận 9', 'Q. 9', 'Q.9', 'quận 9', 'Quận\xa09', 'Quận 9 TP Thủ Đức', 'Quận 9) Thành Phố Thủ Đức'],
    'quận 10': ['Quận 10', 'QUận 10', 'Quận\xa010', 'Q.10', 'quận 10', 'Q. 10', 'Q.10 TPHCM', 'Quận 10)', 'Quận 10 TP. HCM', 'Q.10 Tp HCM', 'quận 10 HCM', 'Quận 10. TP.HCM', 'Quận 10 Hồ Chí Minh', 'q.10'],
    'quận 11': ['Quận 11', 'quận 11', 'Q.11', 'Q. 11', 'q.11'],
    'quận 12': ['Quận 12', 'Q. 12', 'Q.12', 'quận 12', 'Quận 12 Hồ Chí Minh', 'Quận 12 Thành phố Hồ Chí Minh', 'Quận\xa012', 'Q.12 HCM', 'QUận 12', 'quận 12\x08', 'Quận  12'],
    'quận bình thạnh': ['Quận Bình Thành', 'Quận\xa0Bình\xa0Thạnh', 'Quận Bình Thanh','Quận Binh Thạnh', 'Quận Bình Thạnh','Quận Bình Thạnh', 'Q.Bình Thạnh', 'quận Bình Thạnh', 'Q. Bình Thạnh', 'Quận Bình Thạnh TP.Hồ Chí Minh', 'Q. Bình Thạnh TPHCM', 'Quận  Bình Thạnh', 'quận bình thạnh', 'Q.Bình Thạnh TP.HCM', 'Quận Bình Thạnh Hồ Chí Minh', 'Quận Bình Thạnh. TP.HCM', 'Quận Bình Thạnh HCM', 'Quận Bình Thạnh. Hồ Chí Minh', 'QUận Bình Thạnh', 'Quận\xa0Bình Thạnh', 'Quận bình Thạnh', 'Quận\tBình Thạnh', 'Quận.Bình Thạnh', 'Quận Bình Thạnh. Thành phố Hồ Chí Minh'],
    'quận phú nhuận': ['Quận Phú Nhuận', 'Quận\xa0Phú\xa0Nhuận', 'Quận Phú Nhuận', 'Quận Phú Nhuận', 'Quận Phú Nhuận', 'quận Phú Nhuận', 'Q.Phú Nhuận', 'Q. Phú Nhuận', 'Q.Phú Nhuận TPHCM', 'Quận Phú nhuận', 'Q. Phú nhuận', 'Quận Phú Nhuận.TP.HCM', 'Quận Phú Nhuận. Hồ Chí Minh', 'Quận Phú Nhuận Hồ Chí Minh', 'QUẬN PHÚ NHUẬN'],
    'quận gò vấp': ['Quận Gò Vấp','Quận Gò\tVấp','Q.GV','Quận\xa0Gò\xa0Vấp','Quận Gò Vấp', 'quận Gò Vấp', 'Q.Gò Vấp', 'Q. Gò Vấp', 'Q.Gò Vấp Tp.HCM', 'Quận\xa0Gò Vấp', 'Q. Gò Vấp TPHCM', 'Quận Gò Vấp Hồ Chí Minh', 'Quận gò Vấp', 'Quận Gò Vấp TP', 'Quận Gò Vấp HCM', 'Quận Gò vấp', 'Quận  Gò Vấp', 'quận Gò vấp'],
    'quận tân bình': ['Quận tận bình', 'Quận Tân Bình', 'Quận Tân Bı̀nh', 'Quận Tân Binh','Quận Tân Bình', 'quận Tân Bình', 'Q. Tân Bình', 'Q.Tân Bình', 'Quận Tân Bình Hồ Chí Minh', 'Q.Tân bình', 'Q.uận Tân Bình', 'quận Tân bình', 'Quận\xa0Tân Bình', 'quận tân bình TP HCM', 'Quận Tân BÌnh', 'Quận tân Bình', 'Quận Tân bình', 'QuậnTân Bình'],
    'quận tân phú': ['Quận\xa0Tân\xa0Phú','Q.Tân Phú', 'Quận Tận Phú', 'Quận Tân Phú', 'Q.Tân Phú', 'Q. Tân Phú', 'quận Tân Phú', 'Quận\xa0Tân Phú', 'Quận. Tân Phú', 'quận Tân Phú HCM', 'q.Tân phú', 'Quận  Tân Phú', 'Quận Tân Phú Hồ Chí Minh', 'Quận Tân Phú. TPHCM', 'Quận Tân Phú. Hồ Chí Minh', 'Quận Tân phú'],
    'quận bình tân': ['Quận Bình Tân','Quận Bình Trị Đông B','Quận Bình Tân', 'Q. Bình Tân', 'Q.Bình Tân', 'quận Bình Tân', 'Quận\xa0Bình Tân', 'Quận bình Tân', 'Quận Bình Tân (Tầng trệt siêu thị Co.opMart Bình Tân)', 'Quận Quận Bình Tân', 'Quận bình Tân HCM', 'Quận Bình Tân Hồ Chí Minh'],
    'thành phố thủ đức': ['Thành phố Thủ Đức', 'Thành Phố Thủ Đức', 'TP. Thủ Đức', 'TP.Thủ Đức', 'Q.Thủ Đức', 'Quận Thủ Đức', 'TP Thủ Đức', 'quận Thủ Đức', 'Q. Thủ Đức', 'Tp. Thủ Đức', 'Tp.Thủ Đức', 'Tp Thủ Đức', 'tp Thủ Đức', 'thành phố Thủ Đức', 'Thành Phố Thủ ĐỨc', 'Quận Thủ Đức Hồ Chí Minh', 'Thành phố thủ Đức', 'Quận 9 TP Thủ Đức', 'Thành Phố thủ Đức', 'tp thủ đức', 'Thành Phố Thủ đức', 'Thành Phố  Thủ Đức', 'Quận 2 cũ) Thành phố Thủ Đức', 'Quận 9) Thành Phố Thủ Đức', 'quận TP.Thủ Đức', 'Quận 2) Thành Phố Thủ Đức', 'Thành PhốThủ Đức'],
    'quận nhà bè': ['quận nhà bè','Quận Nhà Bè'],
    'quận bình chánh': ['quận bình chánh','Quận Bình Chánh'],
    'quận hóc môn': ['quận hóc môn','Quận Hóc Môn'],
    'quận tân quý' :['Quận Tân Quý'],
    
    # Hà Nội
    'quận ba đình': ['Quận\xa0Ba\xa0Đình', 'Quận Bà Đình','Quận Ba Đình', 'Q. Ba Đình', 'Q.Ba Đình', 'Quận Ba ĐÌnh', 'quận ba đình', 'quận Ba Đình', 'Q.  Ba Đình', 'Quận Ba Đình Hà Nội', 'QUận Ba Đình'],
    'quận hoàn kiếm': ['Quận Hoàng Kiếm','Quận Hoàn Kiếm', 'Q. Hoàn Kiếm', 'quận Hoàn Kiếm', 'Q.Hoàn Kiếm', 'QUận Hoàn Kiếm', 'Quận Hoàn kiếm'],
    'quận đống đa': ['Quận Đồng Đa', 'Quận Đông Đa','Quận\xa0Đống\xa0Đa','Quận Đống Đa', 'quận Đống Đa', 'Q. Đống Đa', 'Q.Đống Đa', 'Quận Đống Đa Hà Nội'],
    'quận hai bà trưng': ['Quận Hai Bà Trưng','Quận Hai Bà Trưng', 'Q. Hai Bà Trưng', 'Q.Hai Bà Trưng', 'quận Hai Bà Trưng', 'Quận Hai Bà Trưng Hà Nội', 'q. Hai Bà Trưng', 'Quận Hai bà Trưng', 'Quận\xa0Hai Bà Trưng', 'quận Hai Bà Trưng Tp. Hà Nội'],
    'quận thanh xuân': ['Quận Thanh Xuân', 'Q. Thanh Xuân', 'Q.Thanh Xuân', 'quận thanh xuân', 'quận Thanh Xuân', 'Quận\xa0Thanh Xuân', 'Quận thanh Xuân', 'Quận Thanh xuân'],
    'quận cầu giấy': ['Quận\xa0Cầu Giấy','Quận Cầu Giấy','Quận Cầu Gấy', 'Q.Cầu Giầy','Quận Cầu Giấy', 'Q.Cầu Giấy', 'Q. Cầu Giấy', 'quận Cầu Giấy', 'Quận Cầu GIấy', 'Quận Cấu Giấy', 'Quận Cầu giấy', 'Quận Cầu Giấy Hà Nội', 'Quận Cần Giấy'],
    'quận tây hồ': ['Quận Tây Hổ','Quận Tây Hồ', 'Q.Tây Hồ', 'Q. Tây Hồ', 'quận Tây Hồ', 'Quận Tây Hồ Hà Nội'],
    'quận hà đông': ['Quận Hà Đồng', 'Quận Hà Đông','Quận Đông Hà','Quận Hà Đông', 'Q. Hà Đông', 'Q.Hà Đông', 'quận Hà Đông', 'Quận  Hà Đông', 'Quận\xa0Hà Đông', 'QuậnHà Đông', 'Quận Hà đông'],
    'quận hoàng mai': ['Quận Hoàn Mai', 'Quận Hoàng Mai', 'Q. Hoàng Mai', 'Q.Hoàng Mai', 'quận Hoàng Mai', 'quận Hoàng mai', 'Quận Hoàng Mai Hà Nội', 'q. Hoàng Mai', 'Quận\xa0Hoàng Mai', 'Quận Quận Hoàng Mai'],
    'quận từ liêm' :['Quận Từ Liêm','Q. Từ Liêm'],
    'quận bắc từ liêm': ['quận bắc từ liêm','Quận Bắc Từ Liêm', 'Q. Bắc Từ Liêm', 'Q.Bắc Từ Liêm', 'quận Bắc Từ Lêm', 'Quận Bắc Từ  Liêm', 'quận Bắc Từ Liêm', 'quận Bắc Từ Liêm Hà Nội', 'Quận Bắc Từ Liên', 'Quận Bắc từ Liêm', 'Quận Bắc Từ Liêm Hà Nội'],
    'quận nam từ liêm': ['Quận Năm Từ Liêm', 'Q. Nam Từ Liêm', 'Quận Nam Từ Liêm', 'Q.Nam Từ Liêm', 'quận Nam Từ Liêm', 'Quận Nam Từ liêm', 'quận Nam Từ LIêm', 'quận nam từ liêm', 'Quận\xa0Nam\xa0Từ\xa0Liêm', 'Quận Nam Từ Liêm Hà Nội', 'Quận Nam Từ Liêm', 'Quận Nam từ Liêm', 'Quận Nam Từ Niêm', 'Quận  Nam Từ Liêm', 'Quận Nam từ liêm'],
    'quận long biên': ['Quận Long Biên', 'Q.Long Biên', 'Q. Long Biên', 'Quận Long biên', 'quận Long Biên', 'Quận\xa0Long Biên'],
    'quận thanh trì': ['quận thanh trì', 'Quận Thanh Trì', 'Quận Thanh Trì', 'quận Thanh Trì', 'Quận Thành Trì'],
    'quận hoài đức' : ['Quận Hoài Đức'],
    'quận gia lâm' :['Q. Gia Lâm'],
    'quận dương nội': ['Quận Dương Nội', 'Q.Dương Nội', 'quận Dương Nội', 'Quận Dương nội'],

}

In [ ]:
flat_normalize_map = {}

for standard, variants in normalize_map_full.items():
    for v in variants:
        flat_normalize_map[v] = standard
    flat_normalize_map[standard] = standard  # Map luôn chuẩn vào chính nó
befood_restaurant["district_mapped"] = befood_restaurant["district_simple"].replace(flat_normalize_map)

In [ ]:
befood_restaurant["district_mapped"].unique()

In [ ]:
df = befood_restaurant.copy()

# Tách known và unknown
known = df[df['district_mapped'].notna()].reset_index(drop=True)
unknown = df[df['district_mapped'].isna()].reset_index()

# Tọa độ
X_known   = known[['latitude','longitude']].values
X_unknown = unknown[['latitude','longitude']].values

# Fit NearestNeighbors (lấy 3 neighbor)
nn = NearestNeighbors(n_neighbors=3, algorithm='ball_tree')
nn.fit(X_known)

# Tìm index 3 neighbors gần nhất
distances, indices = nn.kneighbors(X_unknown)

assigned = []
for idx_list in indices:
    neighbor_districts = known.loc[idx_list, 'district_mapped'].values
    # Lấy district phổ biến nhất
    if len(neighbor_districts) > 0:
        majority_vote = pd.Series(neighbor_districts).mode().iloc[0]
        assigned.append(majority_vote)
    else:
        assigned.append(None)

df.loc[unknown['index'], 'district_mapped'] = assigned
befood_restaurant['district_mapped'] = df['district_mapped']

print("Null còn lại:", befood_restaurant['district_mapped'].isna().sum())
print(befood_restaurant.loc[befood_restaurant['district_mapped'].isna(), 
                            ['address','latitude','longitude']].head(10))

In [ ]:
# Lấy danh sách district duy nhất đã chuẩn hóa
district_list = sorted(df['district_mapped'].unique())

# Gán số thứ tự
district2id = {d: idx for idx, d in enumerate(district_list)}

# Tạo thêm cột district_id
df['district_id'] = df['district_mapped'].map(district2id)

In [ ]:
df.to_csv(r"./Grab/data/data_grb/befood/befood_restaurants.csv", index=False, encoding='utf-8')

## review

In [ ]:
# Tạo dict mapping từng user_name -> thứ tự incremental
unique_users = befood_review['user_name'].unique()
user_map = {name: idx for idx, name in enumerate(unique_users, 1)}

# Áp mapping vào cột mới
befood_review['user_id'] = befood_review['user_name'].map(user_map)
befood_review['platform_id'] = 0

## dishes

In [ ]:
befood_dishes = befood_dishes[['restaurant_id', 'category_id', 'category_name', 'restaurant_item_id', 'item_name', 'price']]
#rename  'restaurant_item_id': 'dish_id'
befood_dishes.rename(columns={'restaurant_item_id': 'dish_id'}, inplace=True)
# add platform_id = 0
befood_dishes['platform_id'] = 0

# Foody

In [ ]:
foody_restaurant = r"./Grab/data/data_grb/foody/restaurants.csv"
foody_review = r"./Grab/data/data_grb/foody/reviews.csv"
foody_dishes = r"./Grab/data/data_grb/foody/dishes.csv"

foody_restaurant = pd.read_csv(foody_restaurant, encoding='utf-8')
foody_review = pd.read_csv(foody_review, encoding='utf-8')
foody_dishes = pd.read_csv(foody_dishes, encoding='utf-8')

## restaurant

In [ ]:
# show 'restaurant_id', 'restaurant_name', 'latitude', 'longitude', 'display_address', 'rating', 'review_count' 
foody_restaurant = foody_restaurant[['restaurant_id', 'restaurant_name', 'latitude', 'longitude', 'address', 'rating', 'review_count']]
# rename columns  'display_address': 'address', 'rating': 'restaurant_rating'
foody_restaurant.rename(columns={'rating': 'restaurant_rating'}, inplace=True)
# scale restaurant_id to band 5 
foody_restaurant['restaurant_rating'] = ((foody_restaurant['restaurant_rating']) / 2).round()
# add platform_id = 1
foody_restaurant['platform_id'] = 1
foody_restaurant.head(3)

In [ ]:
# show distribution of rating
foody_restaurant['restaurant_rating'].value_counts().sort_index().plot(kind='bar')

In [ ]:
# Tách city
def extract_city(address):
    if isinstance(address, str):
        parts = [p.strip() for p in address.split(',') if p.strip()]
        return parts[-1] if parts else None
    return None

# Tách quận/huyện/thành phố trực thuộc tỉnh
def extract_district_raw(address):
    if not isinstance(address, str):
        return None
    parts = [p.strip() for p in address.split(',') if p.strip()]
    if len(parts) < 2:
        return None
    return parts[-2]  # phần trước thành phố

# Áp dụng vào DataFrame
foody_restaurant['city'] = foody_restaurant['address'].apply(extract_city)
foody_restaurant['district_raw'] = foody_restaurant['address'].apply(extract_district_raw)

In [ ]:
# Chuẩn hóa function
def normalize_text(text):
    if isinstance(text, str):
        text = re.sub(r"[\s\u00a0\u200b]+", " ", text)
        text = text.replace(".", "").strip().lower()
        return text
    return text

# Chuẩn hóa city, district
foody_restaurant['city_clean'] = foody_restaurant['city'].apply(normalize_text)
foody_restaurant['district_clean'] = foody_restaurant['district_raw'].apply(normalize_text)

# Tạo list unique
city_list = sorted(foody_restaurant['city_clean'].dropna().unique())
district_list = sorted(foody_restaurant['district_clean'].dropna().unique())

# Gán ID
city_map = {city: idx + 1 for idx, city in enumerate(city_list)}
district_map = {district: idx + 1 for idx, district in enumerate(district_list)}

# Map ID vào DataFrame
foody_restaurant['city_id'] = foody_restaurant['city_clean'].map(city_map)
foody_restaurant['district_id'] = foody_restaurant['district_clean'].map(district_map)

In [ ]:
foody_restaurant.to_csv(r"./Grab/data/data_grb/foody/foody_restaurants.csv", index=False, encoding='utf-8')

## review

In [ ]:
def merge_broken_rows(df):
    merged_rows = []
    current_row = None

    for idx, row in df.iterrows():
        if pd.isna(row['user_id']) and pd.isna(row['user_name']) and pd.isna(row['rating']):
            if current_row is not None:
                current_row['review_text'] = (current_row['review_text'] or '') + ' ' + (row['review_id'] or '')
        else:
            if current_row is not None:
                merged_rows.append(current_row)
            current_row = row.copy()

    if current_row is not None:
        merged_rows.append(current_row)

    return pd.DataFrame(merged_rows)

df_fixed = merge_broken_rows(df)

print(df_fixed.shape)
df_fixed.head()

In [ ]:
df_fixed.dropna(subset=['rating', 'review_text','restaurant_id'], inplace=True)

In [ ]:
#show 'review_id', 'restaurant_id', 'user_id', 'user_name', 'rating', 'review_text', 'created_date'
foody_review = df_fixed[['review_id', 'restaurant_id', 'user_id', 'user_name', 'rating', 'review_text', 'created_date']]
# rename columns 'review_id': 'rating_id', 'created_date': 'review_time'
foody_review.rename(columns={'review_id': 'rating_id', 'created_date': 'review_time'}, inplace=True)
# add platform_id = 1
foody_review['platform_id'] = 1

In [ ]:
# scale rating to band 5
def custom_round(x):
    if pd.isna(x):    # nếu x là NaN thì bỏ qua
        return x
    try:
        return math.ceil(float(x) / 2 - 0.45)
    except:
        return x    # nếu lỗi thì giữ nguyên

# Áp dụng
foody_review['rating'] = foody_review['rating'].apply(custom_round)

foody_review.head(5)

In [ ]:
foody_review.to_csv(r"./Grab/data/data_grb/foody/foody_reviews.csv", index=False, encoding='utf-8')

## dishes